In [1]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
sns.set()

In [2]:
#loading the dataset
tf.keras.datasets.cifar10.load_data() 


170508288/170498071 [==============================] - 2s 0us/step


((array([[[[ 59,  62,  63],
           [ 43,  46,  45],
           [ 50,  48,  43],
           ...,
           [158, 132, 108],
           [152, 125, 102],
           [148, 124, 103]],
  
          [[ 16,  20,  20],
           [  0,   0,   0],
           [ 18,   8,   0],
           ...,
           [123,  88,  55],
           [119,  83,  50],
           [122,  87,  57]],
  
          [[ 25,  24,  21],
           [ 16,   7,   0],
           [ 49,  27,   8],
           ...,
           [118,  84,  50],
           [120,  84,  50],
           [109,  73,  42]],
  
          ...,
  
          [[208, 170,  96],
           [201, 153,  34],
           [198, 161,  26],
           ...,
           [160, 133,  70],
           [ 56,  31,   7],
           [ 53,  34,  20]],
  
          [[180, 139,  96],
           [173, 123,  42],
           [186, 144,  30],
           ...,
           [184, 148,  94],
           [ 97,  62,  34],
           [ 83,  53,  34]],
  
          [[177, 144, 116],
           [16

In [3]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

In [4]:
#check the shape of train and test images. CNN requires 3D images.
#To save on memory you can reshape to a gray scale image by substituting 3 and 1
x=train_images.shape
y=test_images.shape
x,y

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [5]:
#Normalize the data to make sure that all dimensions are on the same scale
train_images_n=train_images/255.
test_images_n=test_images/255.
train_images_n

array([[[[0.23137255, 0.24313725, 0.24705882],
         [0.16862745, 0.18039216, 0.17647059],
         [0.19607843, 0.18823529, 0.16862745],
         ...,
         [0.61960784, 0.51764706, 0.42352941],
         [0.59607843, 0.49019608, 0.4       ],
         [0.58039216, 0.48627451, 0.40392157]],

        [[0.0627451 , 0.07843137, 0.07843137],
         [0.        , 0.        , 0.        ],
         [0.07058824, 0.03137255, 0.        ],
         ...,
         [0.48235294, 0.34509804, 0.21568627],
         [0.46666667, 0.3254902 , 0.19607843],
         [0.47843137, 0.34117647, 0.22352941]],

        [[0.09803922, 0.09411765, 0.08235294],
         [0.0627451 , 0.02745098, 0.        ],
         [0.19215686, 0.10588235, 0.03137255],
         ...,
         [0.4627451 , 0.32941176, 0.19607843],
         [0.47058824, 0.32941176, 0.19607843],
         [0.42745098, 0.28627451, 0.16470588]],

        ...,

        [[0.81568627, 0.66666667, 0.37647059],
         [0.78823529, 0.6       , 0.13333333]

In [6]:
#we have train and test data. split the train data further to train/validation datasets
val_images,train_images=train_images_n[:5000],train_images_n[5000:]
val_labels,train_labels=train_labels[:5000],train_labels[5000:]
test_images=test_images_n

In [7]:
pip install keras-tuner --upgrade

     |████████████████████████████████| 133 kB 5.1 MB/s 


In [8]:
# import the necessary packages
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
import keras_tuner as kt
from tensorflow import keras


In [9]:
x_train=train_images.reshape(-1,32,32,3)
x_test=test_images.reshape(10000,32,32,3)

In [10]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [11]:
def build_model(hp):  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()
    
    model.add(Conv2D(hp.Int('input_units',
                                min_value=32,
                                max_value=1000,
                                step=32), (3, 3), input_shape=x_train.shape[1:]))
    
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for i in range(hp.Int('conv_layers', 1, 4)):  # adding variation of layers.
        model.add(Conv2D(hp.Int(f'conv_{i}_units',
                                min_value=32,
                                max_value=1000,
                                step=32), (3, 3)))
        model.add(Activation('relu'))
    model.add(Flatten()) 
    def build_model(hp):
       for i in range(hp.Int('nn_layers', 1, 4)):  # adding variation of layers.
        model.add(Conv2D(hp.Int(f'neurons',
                                min_value=32,
                                max_value=1000,
                                step=32), (3, 3)))
        model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation("softmax"))

    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

In [12]:
#instantiate the tuner
tuner = kt.Hyperband(hypermodel=build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='CNN HYPERBAND TUNING')

In [13]:
#a callback to stop training early after reaching a certain value for the validation loss
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

In [14]:
tuner.search(x_train, train_labels, epochs=10, callbacks=[stop_early],validation_data=(val_images, val_labels))

Trial 30 Complete [00h 31m 07s]
val_accuracy: 0.7730000019073486

Best val_accuracy So Far: 0.7730000019073486
Total elapsed time: 03h 55m 18s
INFO:tensorflow:Oracle triggered exit


In [15]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the densely-connected
layer is {best_hps.get('nn_layers')}, The optimal number of conv units is {best_hps.get('conv_layers')}  and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



KeyError: ignored

In [16]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, train_labels, epochs=10, validation_data=(val_images, val_labels))

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/10
1407/1407 [==============================] - 184s 130ms/step - loss: 1.5040 - accuracy: 0.4544 - val_loss: 1.2361 - val_accuracy: 0.5724
Epoch 2/10
1407/1407 [==============================] - 184s 131ms/step - loss: 1.0819 - accuracy: 0.6213 - val_loss: 0.9538 - val_accuracy: 0.6730
Epoch 3/10
1407/1407 [==============================] - 183s 130ms/step - loss: 0.8722 - accuracy: 0.6968 - val_loss: 0.8378 - val_accuracy: 0.7102
Epoch 4/10
1407/1407 [==============================] - 183s 130ms/step - loss: 0.7260 - accuracy: 0.7467 - val_loss: 0.7984 - val_accuracy: 0.7178
Epoch 5/10
1407/1407 [==============================] - 183s 130ms/step - loss: 0.6151 - accuracy: 0.7856 - val_loss: 0.7232 - val_accuracy: 0.7590
Epoch 6/10
1407/1407 [==============================] - 183s 130ms/step - loss: 0.5132 - accuracy: 0.8221 - val_loss: 0.7106 - val_accuracy: 0.7580
Epoch 7/10
1407/1407 [==============================] - 183s 130ms/step - loss: 0.4095 - accuracy: 0.8568 - val_

In [18]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_train, train_labels, epochs=best_epoch, validation_data=(val_images, val_labels))

Epoch 1/9
1407/1407 [==============================] - 185s 131ms/step - loss: 1.5096 - accuracy: 0.4536 - val_loss: 1.2333 - val_accuracy: 0.5586
Epoch 2/9
1407/1407 [==============================] - 186s 132ms/step - loss: 1.0887 - accuracy: 0.6152 - val_loss: 0.9535 - val_accuracy: 0.6698
Epoch 3/9
1407/1407 [==============================] - 183s 130ms/step - loss: 0.8791 - accuracy: 0.6939 - val_loss: 0.8651 - val_accuracy: 0.7018
Epoch 4/9
1407/1407 [==============================] - 183s 130ms/step - loss: 0.7323 - accuracy: 0.7461 - val_loss: 0.7630 - val_accuracy: 0.7342
Epoch 5/9
1407/1407 [==============================] - 183s 130ms/step - loss: 0.6174 - accuracy: 0.7866 - val_loss: 0.7130 - val_accuracy: 0.7576
Epoch 6/9
1407/1407 [==============================] - 182s 130ms/step - loss: 0.5138 - accuracy: 0.8228 - val_loss: 0.6827 - val_accuracy: 0.7662
Epoch 7/9
1407/1407 [==============================] - 182s 130ms/step - loss: 0.4080 - accuracy: 0.8590 - val_loss: 0

In [19]:
eval_result = hypermodel.evaluate(test_images, test_labels)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 15s 47ms/step - loss: 0.8472 - accuracy: 0.7527
[test loss, test accuracy]: [0.8472378849983215, 0.7526999711990356]
